In [1]:
!pip install -U sentence-transformers
!pip install - q cupy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.1 MB/s eta 0:00:0000:01
ERROR: Invalid requirement: '-'


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import cupy as cp
import numpy as np

### Import data

In [3]:
df = pd.read_csv('/kaggle/input/zaloai2021-legal-text-retrieval/legal_corpus_original.csv') #using splited version for better encoding
df.head()

,law_title,article_id,article_title,clause_id,point_id,content,content_length
0,01/2009/tt-bnn,1,Điều 1. Phạm vi áp dụng,0,0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",37
1,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,0,0,NaN,1
2,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,1,0,"\tKhoản 1. Hàng năm trước mùa mưa, lũ, Ủy ban ...",86
3,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,2,0,"\tKhoản 2. Lực lượng tuần tra, canh gác đê đượ...",80
4,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,3,0,"\tKhoản 3. Khi lũ, bão có diễn biến phức tạp, ...",36


### Preprocessing data

In [4]:
df = df.dropna(subset=['content'])
df.head()

,law_title,article_id,article_title,clause_id,point_id,content,content_length
0,01/2009/tt-bnn,1,Điều 1. Phạm vi áp dụng,0,0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",37
2,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,1,0,"\tKhoản 1. Hàng năm trước mùa mưa, lũ, Ủy ban ...",86
3,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,2,0,"\tKhoản 2. Lực lượng tuần tra, canh gác đê đượ...",80
4,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,3,0,"\tKhoản 3. Khi lũ, bão có diễn biến phức tạp, ...",36
6,01/2009/tt-bnn,3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,1,0,"\tKhoản 1. Là người khoẻ mạnh, tháo vát, đủ kh...",27


In [5]:
df['id'] = [i for i in range(len(df))]
df = df[:10]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 12
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   law_title       10 non-null     object
 1   article_id      10 non-null     int64 
 2   article_title   10 non-null     object
 3   clause_id       10 non-null     int64 
 4   point_id        10 non-null     int64 
 5   content         10 non-null     object
 6   content_length  10 non-null     int64 
 7   id              10 non-null     int64 
dtypes: int64(5), object(3)
memory usage: 720.0+ bytes


In [6]:
df.loc[:, 'law_article'] = df.apply(lambda row: {'law_id': row['law_title'], 'article_id': str(row['article_id'])}, axis=1)
df.head()

,law_title,article_id,article_title,clause_id,point_id,content,content_length,id,law_article
0,01/2009/tt-bnn,1,Điều 1. Phạm vi áp dụng,0,0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",37,0,"{'law_id': '01/2009/tt-bnn', 'article_id': '1'}"
2,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,1,0,"\tKhoản 1. Hàng năm trước mùa mưa, lũ, Ủy ban ...",86,1,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}"
3,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,2,0,"\tKhoản 2. Lực lượng tuần tra, canh gác đê đượ...",80,2,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}"
4,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,3,0,"\tKhoản 3. Khi lũ, bão có diễn biến phức tạp, ...",36,3,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}"
6,01/2009/tt-bnn,3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,1,0,"\tKhoản 1. Là người khoẻ mạnh, tháo vát, đủ kh...",27,4,"{'law_id': '01/2009/tt-bnn', 'article_id': '3'}"


### Document Encoding

In [7]:
# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentences = ["Cô ấy là một người vui_tính .", "Cô ấy cười nói suốt cả ngày ."]

model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
embeddings = model.encode(sentences)
print(embeddings)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[ 0.11881631 -0.32826942 -0.28465226 ... -0.3070276  -0.27665684
  -0.03120276]
 [ 0.33210808 -0.17807022 -0.39799985 ... -0.33134738 -0.44188038
  -0.18214273]]


In [8]:
def encode(lst = [], convert_to_tensor=True):
    encoded_vectors = model.encode(lst, convert_to_tensor=True)
    encoded_vectors_cpu = cp.asnumpy(encoded_vectors)  # Convert to NumPy array
    vector_arr = [np.array(arr) for arr in encoded_vectors.cpu().detach().numpy()]
    return vector_arr

In [9]:
df['vector'] = encode(lst = list(df['content']))
df.head()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,law_title,article_id,article_title,clause_id,point_id,content,content_length,id,law_article,vector
0,01/2009/tt-bnn,1,Điều 1. Phạm vi áp dụng,0,0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",37,0,"{'law_id': '01/2009/tt-bnn', 'article_id': '1'}","[0.19059409, 0.03537087, 0.03364963, -0.038163..."
2,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,1,0,"\tKhoản 1. Hàng năm trước mùa mưa, lũ, Ủy ban ...",86,1,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}","[0.14171001, 0.05883229, 0.03424324, 0.3469323..."
3,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,2,0,"\tKhoản 2. Lực lượng tuần tra, canh gác đê đượ...",80,2,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}","[0.00016879971, -0.033614866, 0.07556901, 0.28..."
4,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,3,0,"\tKhoản 3. Khi lũ, bão có diễn biến phức tạp, ...",36,3,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}","[0.071038745, 0.027476337, -0.11718064, 0.2727..."
6,01/2009/tt-bnn,3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,1,0,"\tKhoản 1. Là người khoẻ mạnh, tháo vát, đủ kh...",27,4,"{'law_id': '01/2009/tt-bnn', 'article_id': '3'}","[0.07951737, 0.12896462, -0.15411736, 0.044079..."


In [10]:
df.to_json('/kaggle/working/legal_copus_data.json')

In [11]:
df = pd.read_json('/kaggle/working/legal_copus_data.json')
df

,law_title,article_id,article_title,clause_id,point_id,content,content_length,id,law_article,vector
0,01/2009/tt-bnn,1,Điều 1. Phạm vi áp dụng,0,0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",37,0,"{'law_id': '01/2009/tt-bnn', 'article_id': '1'}","[0.19059409200000002, 0.035370871400000004, 0...."
2,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,1,0,"\tKhoản 1. Hàng năm trước mùa mưa, lũ, Ủy ban ...",86,1,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}","[0.1417100132, 0.0588322915, 0.034243241, 0.34..."
3,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,2,0,"\tKhoản 2. Lực lượng tuần tra, canh gác đê đượ...",80,2,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}","[0.00016879970000000002, -0.0336148664, 0.0755..."
4,01/2009/tt-bnn,2,Điều 2. Tổ chức lực lượng,3,0,"\tKhoản 3. Khi lũ, bão có diễn biến phức tạp, ...",36,3,"{'law_id': '01/2009/tt-bnn', 'article_id': '2'}","[0.0710387453, 0.0274763368, -0.117180638, 0.2..."
6,01/2009/tt-bnn,3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,1,0,"\tKhoản 1. Là người khoẻ mạnh, tháo vát, đủ kh...",27,4,"{'law_id': '01/2009/tt-bnn', 'article_id': '3'}","[0.079517372, 0.12896461780000001, -0.15411736..."
7,01/2009/tt-bnn,3,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,2,0,"\tKhoản 2. Có tinh thần trách nhiệm, chịu đựng...",32,5,"{'law_id': '01/2009/tt-bnn', 'article_id': '3'}","[0.070066683, 0.1165396273, -0.077451996500000..."
9,01/2009/tt-bnn,4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",1,0,\tKhoản 1. Chấp hành sự phân công của Ban chỉ ...,36,6,"{'law_id': '01/2009/tt-bnn', 'article_id': '4'}","[0.135596022, 0.1884660572, 0.0952912048, 0.13..."
10,01/2009/tt-bnn,4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",2,0,"\tKhoản 2. Tuần tra, canh gác và thường trực t...",89,7,"{'law_id': '01/2009/tt-bnn', 'article_id': '4'}","[0.23750269410000002, 0.16307845710000002, -0...."
11,01/2009/tt-bnn,4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",3,0,\tKhoản 3. Tham gia xử lý sự cố và tu sửa kịp ...,43,8,"{'law_id': '01/2009/tt-bnn', 'article_id': '4'}","[-0.0760755241, 0.2262742519, 0.09843015670000..."
12,01/2009/tt-bnn,4,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",4,0,"\tKhoản 4. Canh gác, kiểm tra phát hiện và ngă...",40,9,"{'law_id': '01/2009/tt-bnn', 'article_id': '4'}","[0.10587478430000001, 0.161282599, -0.08721502..."


### Train & Test

In [12]:
df_train_test = pd.read_csv('/kaggle/input/zaloai2021-legal-text-retrieval/train_qna.csv')
from sklearn.model_selection import train_test_split
import pandas as pd

# Assume df is your DataFrame
# Splitting the DataFrame into 80% train and 20% test
train_df, test_df = train_test_split(df_train_test, test_size=0.2, random_state=42)
test_df['id'] = [i for i in range(len(test_df))]
# Printing the sizes of train and test subsets
print("Train subset size:", len(train_df))
print("Test subset size:", len(test_df))

Train subset size: 2556
Test subset size: 640


In [13]:
test_df['vector'] = encode(lst = list(test_df['question']))

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [14]:
test_df.head()

,question_id,question,relevant_articles,question_length,num_articles,relevant_titles,id,vector
1951,8e2cfe626cebf209f94e0db8f147960c,Mức xử phạt đối với hành vi không tổ chức khám...,"[{'law_id': '28/2020/nđ-cp', 'article_id': '21'}]",29,1,['28/2020/nđ-cp'],0,"[-0.021589346, -0.23637885, 0.10553159, -0.068..."
1204,0a32724630653580cc90c77bcf552baf,Tranh chấp giữa cơ quan ký kết hợp đồng dự án ...,"[{'law_id': '64/2020/qh14', 'article_id': '97'}]",27,1,['64/2020/qh14'],1,"[-0.18010645, -0.0015339414, -0.042509265, 0.1..."
2661,e66ae5eecc1672bac2c5799673666bda,"Trình tự cấp, cấp lại Giấy chứng nhận đủ điều ...","[{'law_id': '26/2019/nđ-cp', 'article_id': '28'}]",29,1,['26/2019/nđ-cp'],2,"[0.082680196, -0.35022604, 0.29756254, -0.1278..."
705,6b815a5b10736472101034aa8bd58146,Thành viên hợp danh thì có được làm chủ doanh ...,"[{'law_id': '59/2020/qh14', 'article_id': '180'}]",12,1,['59/2020/qh14'],3,"[-0.07432145, 0.21651307, -0.014255168, -0.319..."
1036,4d4ed6ef19708f3e18fef48e4786a06e,Luật sư có được miễn đào tạo nghề đấu giá hay ...,"[{'law_id': '01/2016/qh14', 'article_id': '12'}]",12,1,['01/2016/qh14'],4,"[-0.06426895, -0.28210092, 0.33683205, -0.1060..."


### Search by comparing similarity

In [15]:
import cupy as cp
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# # Sample query and document dataframes
# queries = pd.DataFrame({'id': [1, 2],
#                         'vector': [[1, 2, 3], [4, 5, 6]]})
# documents = pd.DataFrame({'id': [1, 2, 3, 4],
#                           'vector': [[[1, 2, 3]], [[5, 6, 7]],[[2, 3, 4]],[[8, 9, 10]]]})

def top_k_relevance(queries, documents, k):
    # Calculate similarity matrix using NumPy arrays
    similarity_matrix = cosine_similarity(np.vstack(queries['vector'].to_numpy()),
                                          np.vstack(documents['vector'].to_numpy()))
    
    top_k_matrix = {}
    for i in range(len(queries)):
        query_id = queries['id'].iloc[i]
        similarity_scores = similarity_matrix[i]
        top_k_indices = np.argsort(similarity_scores)[-k:][::-1]
        top_k_doc_ids = documents['id'].iloc[top_k_indices].tolist()
        top_k_matrix[query_id] = top_k_doc_ids

    return top_k_matrix



In [ ]:
top_k = 2
result = top_k_relevance(test_df, df, top_k)
# print(result)

### Postprocess relevant document

In [17]:
test_df['proposed_relevance'] = [[df['law_article'].iloc[i] for i in result[q]] for q in result]


In [18]:
test_df.head()

,question_id,question,relevant_articles,question_length,num_articles,relevant_titles,id,vector,proposed_relevance
1951,8e2cfe626cebf209f94e0db8f147960c,Mức xử phạt đối với hành vi không tổ chức khám...,"[{'law_id': '28/2020/nđ-cp', 'article_id': '21'}]",29,1,['28/2020/nđ-cp'],0,"[-0.021589346, -0.23637885, 0.10553159, -0.068...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4..."
1204,0a32724630653580cc90c77bcf552baf,Tranh chấp giữa cơ quan ký kết hợp đồng dự án ...,"[{'law_id': '64/2020/qh14', 'article_id': '97'}]",27,1,['64/2020/qh14'],1,"[-0.18010645, -0.0015339414, -0.042509265, 0.1...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4..."
2661,e66ae5eecc1672bac2c5799673666bda,"Trình tự cấp, cấp lại Giấy chứng nhận đủ điều ...","[{'law_id': '26/2019/nđ-cp', 'article_id': '28'}]",29,1,['26/2019/nđ-cp'],2,"[0.082680196, -0.35022604, 0.29756254, -0.1278...","[{'law_id': '01/2009/tt-bnn', 'article_id': '1..."
705,6b815a5b10736472101034aa8bd58146,Thành viên hợp danh thì có được làm chủ doanh ...,"[{'law_id': '59/2020/qh14', 'article_id': '180'}]",12,1,['59/2020/qh14'],3,"[-0.07432145, 0.21651307, -0.014255168, -0.319...","[{'law_id': '01/2009/tt-bnn', 'article_id': '2..."
1036,4d4ed6ef19708f3e18fef48e4786a06e,Luật sư có được miễn đào tạo nghề đấu giá hay ...,"[{'law_id': '01/2016/qh14', 'article_id': '12'}]",12,1,['01/2016/qh14'],4,"[-0.06426895, -0.28210092, 0.33683205, -0.1060...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4..."


In [46]:
test_df['proposed_relevance'].iloc[0] = [ {'law_id': '28/2020/nđ-cp', 'article_id': '21'}, {'law_id': '64/2020/qh14', 'article_id': '97'}]

/tmp/ipykernel_34/2728735399.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_df['proposed_relevance'].iloc[0] = [ {'law_id': '28/2020/nđ-cp', 'article_id': '21'}, {'law_id': '64/2020/qh14', 'article_id': '97'}]
/tmp/ipykernel_34/272

## TESTING

In [47]:
import numpy as np

def recall(test_df, top_k):
    assert top_k == len(test_df['proposed_relevance'].iloc[0]), "Wrong top k"
    
    def calculate_recall(row):
        relevant_documents = set([str(e) for e in eval(row['relevant_articles'])])

        retrieved_documents = set([str(i) for i in row['proposed_relevance']])
        return len(relevant_documents.intersection(retrieved_documents)) / len(relevant_documents) if len(relevant_documents) > 0 else 0

    test_df['recall'] = test_df.apply(calculate_recall, axis=1)
    
    return np.mean(test_df['recall'])

recall_score = recall(test_df, top_k)
print("Recall Score:", recall_score)
test_df.head()

Recall Score: 0.0015625


,question_id,question,relevant_articles,question_length,num_articles,relevant_titles,id,vector,proposed_relevance,recall,precision,f1,f2,AP,NDCG
1951,8e2cfe626cebf209f94e0db8f147960c,Mức xử phạt đối với hành vi không tổ chức khám...,"[{'law_id': '28/2020/nđ-cp', 'article_id': '21'}]",29,1,['28/2020/nđ-cp'],0,"[-0.021589346, -0.23637885, 0.10553159, -0.068...","[{'law_id': '28/2020/nđ-cp', 'article_id': '21...",1.0,0.5,0.666667,0.833333,1.0,1.0
1204,0a32724630653580cc90c77bcf552baf,Tranh chấp giữa cơ quan ký kết hợp đồng dự án ...,"[{'law_id': '64/2020/qh14', 'article_id': '97'}]",27,1,['64/2020/qh14'],1,"[-0.18010645, -0.0015339414, -0.042509265, 0.1...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0
2661,e66ae5eecc1672bac2c5799673666bda,"Trình tự cấp, cấp lại Giấy chứng nhận đủ điều ...","[{'law_id': '26/2019/nđ-cp', 'article_id': '28'}]",29,1,['26/2019/nđ-cp'],2,"[0.082680196, -0.35022604, 0.29756254, -0.1278...","[{'law_id': '01/2009/tt-bnn', 'article_id': '1...",0.0,0.0,0.000000,0.000000,0.0,0.0
705,6b815a5b10736472101034aa8bd58146,Thành viên hợp danh thì có được làm chủ doanh ...,"[{'law_id': '59/2020/qh14', 'article_id': '180'}]",12,1,['59/2020/qh14'],3,"[-0.07432145, 0.21651307, -0.014255168, -0.319...","[{'law_id': '01/2009/tt-bnn', 'article_id': '2...",0.0,0.0,0.000000,0.000000,0.0,0.0
1036,4d4ed6ef19708f3e18fef48e4786a06e,Luật sư có được miễn đào tạo nghề đấu giá hay ...,"[{'law_id': '01/2016/qh14', 'article_id': '12'}]",12,1,['01/2016/qh14'],4,"[-0.06426895, -0.28210092, 0.33683205, -0.1060...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0


In [48]:
def precision(test_df, top_k):
    assert top_k == len(test_df['proposed_relevance'].iloc[0]), "Wrong top k"
    
    def calculate_precision(row):
        relevant_documents = set([str(e) for e in eval(row['relevant_articles'])])
        retrieved_documents = set([str(i) for i in row['proposed_relevance']])
        return len(relevant_documents.intersection(retrieved_documents)) / len(retrieved_documents) if len(retrieved_documents) > 0 else 0

    test_df['precision'] = test_df.apply(calculate_precision, axis=1)
    
    return np.mean(test_df['precision'])

precision_score = precision(test_df, top_k)
print("Precision Score:", precision_score)
test_df.head()


Precision Score: 0.00078125


,question_id,question,relevant_articles,question_length,num_articles,relevant_titles,id,vector,proposed_relevance,recall,precision,f1,f2,AP,NDCG
1951,8e2cfe626cebf209f94e0db8f147960c,Mức xử phạt đối với hành vi không tổ chức khám...,"[{'law_id': '28/2020/nđ-cp', 'article_id': '21'}]",29,1,['28/2020/nđ-cp'],0,"[-0.021589346, -0.23637885, 0.10553159, -0.068...","[{'law_id': '28/2020/nđ-cp', 'article_id': '21...",1.0,0.5,0.666667,0.833333,1.0,1.0
1204,0a32724630653580cc90c77bcf552baf,Tranh chấp giữa cơ quan ký kết hợp đồng dự án ...,"[{'law_id': '64/2020/qh14', 'article_id': '97'}]",27,1,['64/2020/qh14'],1,"[-0.18010645, -0.0015339414, -0.042509265, 0.1...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0
2661,e66ae5eecc1672bac2c5799673666bda,"Trình tự cấp, cấp lại Giấy chứng nhận đủ điều ...","[{'law_id': '26/2019/nđ-cp', 'article_id': '28'}]",29,1,['26/2019/nđ-cp'],2,"[0.082680196, -0.35022604, 0.29756254, -0.1278...","[{'law_id': '01/2009/tt-bnn', 'article_id': '1...",0.0,0.0,0.000000,0.000000,0.0,0.0
705,6b815a5b10736472101034aa8bd58146,Thành viên hợp danh thì có được làm chủ doanh ...,"[{'law_id': '59/2020/qh14', 'article_id': '180'}]",12,1,['59/2020/qh14'],3,"[-0.07432145, 0.21651307, -0.014255168, -0.319...","[{'law_id': '01/2009/tt-bnn', 'article_id': '2...",0.0,0.0,0.000000,0.000000,0.0,0.0
1036,4d4ed6ef19708f3e18fef48e4786a06e,Luật sư có được miễn đào tạo nghề đấu giá hay ...,"[{'law_id': '01/2016/qh14', 'article_id': '12'}]",12,1,['01/2016/qh14'],4,"[-0.06426895, -0.28210092, 0.33683205, -0.1060...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0


In [49]:
def f1(test_df, top_k):
    assert top_k == len(test_df['proposed_relevance'].iloc[0]), "Wrong top k"
    
    def calculate_f1(row):
        precision = row['precision']
        recall = row['recall']
        if precision + recall == 0:
            return 0
        return 2 * (precision * recall) / (precision + recall)

    test_df['f1'] = test_df.apply(calculate_f1, axis=1)
    
    return np.mean(test_df['f1'])

def f2(test_df, top_k):
    assert top_k == len(test_df['proposed_relevance'].iloc[0]), "Wrong top k"
    
    def calculate_f2(row):
        precision = row['precision']
        recall = row['recall']
        if precision + recall == 0:
            return 0
        return (5 * precision * recall) / (4 * precision + recall)

    test_df['f2'] = test_df.apply(calculate_f2, axis=1)
    
    return np.mean(test_df['f2'])


f1_score = f1(test_df, top_k)
f2_score = f2(test_df, top_k)

print("F1 Score:", f1_score)
print("F2 Score:", f2_score)


F1 Score: 0.0010416666666666667
F2 Score: 0.0013020833333333335


### mAP 

This metric can not judge which is relevant more or less

In [50]:
def mAP(test_df, top_k):
    def calculateAP(row):
        relevant_documents = set([str(e) for e in eval(row['relevant_articles'])])
        ap_num = 0
        for k in range(1,top_k+1):
            
            retrieved_documents = set([str(i) for i in row['proposed_relevance'][:k]])
            precision_k = len(relevant_documents.intersection(retrieved_documents)) / len(retrieved_documents) if len(retrieved_documents) > 0 else 0
            retrieved = [str(i) for i in row['proposed_relevance']]
            if retrieved[k-1] in relevant_documents:
                rel_k = 1
            else:
                rel_k = 0
            
            ap_num += precision_k * rel_k
        return ap_num/len(relevant_documents)
    
    test_df['AP'] = test_df.apply(calculateAP, axis=1)
    return np.mean(test_df['AP'])


print("mAP: ", mAP(test_df, top_k))
test_df.head()

mAP:  0.0015625


,question_id,question,relevant_articles,question_length,num_articles,relevant_titles,id,vector,proposed_relevance,recall,precision,f1,f2,AP,NDCG
1951,8e2cfe626cebf209f94e0db8f147960c,Mức xử phạt đối với hành vi không tổ chức khám...,"[{'law_id': '28/2020/nđ-cp', 'article_id': '21'}]",29,1,['28/2020/nđ-cp'],0,"[-0.021589346, -0.23637885, 0.10553159, -0.068...","[{'law_id': '28/2020/nđ-cp', 'article_id': '21...",1.0,0.5,0.666667,0.833333,1.0,1.0
1204,0a32724630653580cc90c77bcf552baf,Tranh chấp giữa cơ quan ký kết hợp đồng dự án ...,"[{'law_id': '64/2020/qh14', 'article_id': '97'}]",27,1,['64/2020/qh14'],1,"[-0.18010645, -0.0015339414, -0.042509265, 0.1...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0
2661,e66ae5eecc1672bac2c5799673666bda,"Trình tự cấp, cấp lại Giấy chứng nhận đủ điều ...","[{'law_id': '26/2019/nđ-cp', 'article_id': '28'}]",29,1,['26/2019/nđ-cp'],2,"[0.082680196, -0.35022604, 0.29756254, -0.1278...","[{'law_id': '01/2009/tt-bnn', 'article_id': '1...",0.0,0.0,0.000000,0.000000,0.0,0.0
705,6b815a5b10736472101034aa8bd58146,Thành viên hợp danh thì có được làm chủ doanh ...,"[{'law_id': '59/2020/qh14', 'article_id': '180'}]",12,1,['59/2020/qh14'],3,"[-0.07432145, 0.21651307, -0.014255168, -0.319...","[{'law_id': '01/2009/tt-bnn', 'article_id': '2...",0.0,0.0,0.000000,0.000000,0.0,0.0
1036,4d4ed6ef19708f3e18fef48e4786a06e,Luật sư có được miễn đào tạo nghề đấu giá hay ...,"[{'law_id': '01/2016/qh14', 'article_id': '12'}]",12,1,['01/2016/qh14'],4,"[-0.06426895, -0.28210092, 0.33683205, -0.1060...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0


### Normalized Discounted Cumulative Gain

In [51]:
def get_relevance(relevant_articles:list, proposed_relevance:list):
    '''
    Input: list of full value (not need to be unique). In this case 2 paragraph in same lawid can be same relevance score, also consider relevance to each other while with precision, we dont care 
    Output: relevance = [0, 1, 2, 4, 6, 1, 4, 7]'''
    relevance = []
    for art in proposed_relevance:
        try:
            relevance.append(len(relevant_articles) - relevant_articles.index(art) )
        except ValueError:
            relevance.append(0)
    return relevance
relevant_articles = [3,4]
proposed_relevance = [2,1,3,5,4,5]
get_relevance(relevant_articles, proposed_relevance)


[0, 0, 2, 0, 1, 0]

In [52]:
def NDCG_K(test_df, top_k):
    K = top_k
    
    def calculateNDCG(row):
        relevant_documents = [str(e) for e in eval(row['relevant_articles'])]
        retrieved_documents = [str(i) for i in row['proposed_relevance']]
        relevance = get_relevance(relevant_documents,retrieved_documents)
        ideal_relevance = sorted(relevance, reverse=True)
        dcg = 0
        idcg = 0
        for k in range(1, K+1):
            # calculate rel_k values
            rel_k = relevance[k-1]
            ideal_rel_k = ideal_relevance[k-1]
            # calculate dcg and idcg
            dcg += rel_k / log2(1 + k)
            idcg += ideal_rel_k / log2(1 + k)
            # calcualte ndcg
        ndcg = dcg / idcg if idcg > 0 else 0
        return ndcg
    test_df['NDCG'] = test_df.apply(calculateNDCG, axis=1)
    return np.mean(test_df['NDCG'])
print("NDCG: ", NDCG_K(test_df, top_k))
test_df.head()

NDCG:  0.0015625


,question_id,question,relevant_articles,question_length,num_articles,relevant_titles,id,vector,proposed_relevance,recall,precision,f1,f2,AP,NDCG
1951,8e2cfe626cebf209f94e0db8f147960c,Mức xử phạt đối với hành vi không tổ chức khám...,"[{'law_id': '28/2020/nđ-cp', 'article_id': '21'}]",29,1,['28/2020/nđ-cp'],0,"[-0.021589346, -0.23637885, 0.10553159, -0.068...","[{'law_id': '28/2020/nđ-cp', 'article_id': '21...",1.0,0.5,0.666667,0.833333,1.0,1.0
1204,0a32724630653580cc90c77bcf552baf,Tranh chấp giữa cơ quan ký kết hợp đồng dự án ...,"[{'law_id': '64/2020/qh14', 'article_id': '97'}]",27,1,['64/2020/qh14'],1,"[-0.18010645, -0.0015339414, -0.042509265, 0.1...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0
2661,e66ae5eecc1672bac2c5799673666bda,"Trình tự cấp, cấp lại Giấy chứng nhận đủ điều ...","[{'law_id': '26/2019/nđ-cp', 'article_id': '28'}]",29,1,['26/2019/nđ-cp'],2,"[0.082680196, -0.35022604, 0.29756254, -0.1278...","[{'law_id': '01/2009/tt-bnn', 'article_id': '1...",0.0,0.0,0.000000,0.000000,0.0,0.0
705,6b815a5b10736472101034aa8bd58146,Thành viên hợp danh thì có được làm chủ doanh ...,"[{'law_id': '59/2020/qh14', 'article_id': '180'}]",12,1,['59/2020/qh14'],3,"[-0.07432145, 0.21651307, -0.014255168, -0.319...","[{'law_id': '01/2009/tt-bnn', 'article_id': '2...",0.0,0.0,0.000000,0.000000,0.0,0.0
1036,4d4ed6ef19708f3e18fef48e4786a06e,Luật sư có được miễn đào tạo nghề đấu giá hay ...,"[{'law_id': '01/2016/qh14', 'article_id': '12'}]",12,1,['01/2016/qh14'],4,"[-0.06426895, -0.28210092, 0.33683205, -0.1060...","[{'law_id': '01/2009/tt-bnn', 'article_id': '4...",0.0,0.0,0.000000,0.000000,0.0,0.0


In [1]:
import pandas as pd

df = pd.read_json('/home/LegalDocumentRetrieval-20241027T111633Z-001/LegalDocumentRetrieval/encoded_public_test.json')
df



,question,qid,question_vector
0,Hiệp hội Công nghiệp ghi âm Việt Nam hoạt động...,98440,"[-0.1099399105, 0.3645758927, 0.1657535434, 0...."
1,Báo cáo nghiên cứu khả thi đầu tư xây dựng là ...,105737,"[0.2333178967, -0.0948213562, -0.1287208796, -..."
2,Lịch khai giảng năm học 2022 - 2023 đối với họ...,106239,"[0.055089771700000005, -0.0855046064, -0.12588..."
3,Số định danh cá nhân có được dùng thay thế các...,79491,"[-0.0940591767, 0.1100925282, -0.3543012738000..."
4,Trợ cấp đối với Chủ tịch Hội cựu chiến binh cấ...,130557,"[-0.0505643375, 0.0249973089, 0.0473551191, 0...."
...,...,...,...
9995,Đón trả hành khách trên đường cao tốc có bị gi...,42798,"[-0.0168471839, -0.07561205330000001, -0.24143..."
9996,"Các đơn vị được giao là đầu mối trao đổi, cung...",10533,"[-0.051241323400000004, -0.1234926432, 0.01502..."
9997,Ban Thường vụ Hội Hỗ trợ khắc phục hậu quả bom...,46794,"[-0.0481531881, 0.0365075208, 0.12054245170000..."
9998,"Tài liệu thông tin, giáo dục, truyền thông về ...",112007,"[0.0006734343, -0.46517518160000004, 0.1443595..."


In [ ]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('/home/LegalDocumentRetrieval-20241027T111633Z-001/LegalDocumentRetrieval/train.csv',
                       on_bad_lines='skip')
df_train = df_train.head(1000)

# Load the data

count_retrieve = np.zeros(10)  # Initialize the count array

with open("/home/LegalDocumentRetrieval-20241027T111633Z-001/LegalDocumentRetrieval/predict_train_mini.txt", "r") as f:
    for line in f:
        line = line.strip().split()
        qid = line[0]
        retrieved_cids = line[1:]  # cid_1 to cid_10

        # Get the ground truth cids for this qid from the DataFrame
        ground_truth_cids = df_train[df_train['qid'] == qid]['cid'].tolist()

        # If 'cid' is not a list of strings, but a string representation of a list, convert it:
        if isinstance(ground_truth_cids, str):
             ground_truth_cids = ground_truth_cids.strip('[]').split()
             ground_truth_cids = [cid.strip() for cid in ground_truth_cids]

        # Check each ground truth cid against the retrieved cids
        for true_cid in ground_truth_cids:
            for i, retrieved_cid in enumerate(retrieved_cids):
                if true_cid == retrieved_cid:
                    count_retrieve[i] += 1
                    break  # Move to the next true_cid once it's found

print("Retrieval Rank Counts:", count_retrieve)

# Calculate recall@k
recall_at_1 = count_retrieve[0] / len(df_train)
recall_at_5 = np.sum(count_retrieve[:5]) / len(df_train)
recall_at_10 = np.sum(count_retrieve) / len(df_train)

print(f"Recall@1: {recall_at_1:.4f}")
print(f"Recall@5: {recall_at_5:.4f}")
print(f"Recall@10: {recall_at_10:.4f}")



ParserError: Error tokenizing data. C error: Expected 4 fields in line 23453, saw 5


In [6]:
import pandas as pd
import numpy as np

df_train = pd.read_csv(
    '/home/LegalDocumentRetrieval-20241027T111633Z-001/LegalDocumentRetrieval/train.csv',
    on_bad_lines='skip'  # Skip lines with unexpected number of fields
)
df_train = df_train.head(1000)

# Load the data

count_retrieve = np.zeros(10)  # Initialize the count array

with open("/home/LegalDocumentRetrieval-20241027T111633Z-001/LegalDocumentRetrieval/predict_train_mini.txt", "r") as f:
    for line in f:
        line = line.strip().split()
        qid = line[0]
        retrieved_cids = line[1:]  # cid_1 to cid_10

        # Get the ground truth cids for this qid from the DataFrame
        ground_truth_row = df_train[df_train['qid'] == int(qid)]
        if not ground_truth_row.empty:
            ground_truth_str = ground_truth_row['cid'].iloc[0]
            # Convert "[31682 31677]" format to list of strings
            ground_truth_cids = ground_truth_str.strip('[]').split()
            ground_truth_cids = [str(cid) for cid in ground_truth_cids]

        # Check each ground truth cid against the retrieved cids
        for true_cid in ground_truth_cids:
            for i, retrieved_cid in enumerate(retrieved_cids):
                if true_cid == retrieved_cid:
                    count_retrieve[i] += 1
                    break  # Move to the next true_cid once it's found

print("Retrieval Rank Counts:", count_retrieve)

# Calculate recall@k
recall_at_1 = count_retrieve[0] / len(df_train)
recall_at_5 = np.sum(count_retrieve[:5]) / len(df_train)
recall_at_10 = np.sum(count_retrieve) / len(df_train)

print(f"Recall@1: {recall_at_1:.4f}")
print(f"Recall@5: {recall_at_5:.4f}")
print(f"Recall@10: {recall_at_10:.4f}")



Retrieval Rank Counts: [253. 100.  65.  42.  41.  30.  18.  20.  21.  13.]
Recall@1: 0.2530
Recall@5: 0.5010
Recall@10: 0.6030
